# 동작구 Analysis : 크롤링한 데이터를 다중회귀분석하고 머신러닝으로 학습하기

앞서 수집한 데이터 csv를 불러와서 분석에 알맞게 가공하고, 다중회귀분석(다차원의 x가 존재하는 회귀)을 test,data셋으로 나누어 정확도를 높인다. 사용할 model과 손실함수는 ~

Reference
http://hleecaster.com/ml-multiple-linear-regression-example/

In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np

In [3]:
df1 = pd.read_csv("Dongjak_Bigdata_re.csv")
df2 = pd.read_csv("Dongjak_Bigdata_review_re.csv")

In [4]:
df1 = DataFrame(df1)
df2 = DataFrame(df2)

## 빅데이터 검색결과 크롤링

### csv에서 필요한 부분만 데이터프레임으로 만들기

In [5]:
df1

,Unnamed: 0,place_name,tour_element,tour_element_name
0,0,1.21 무장공비 침투로,음식점,0
1,1,1.21 무장공비 침투로,카페,0
2,2,1.21 무장공비 침투로,숙소,0
3,3,1.21 무장공비 침투로,교통,0
4,4,1.21 무장공비 침투로,공원,0
...,...,...,...,...
22997,22997,목포근대역사관,쇼핑,산마을생식
22998,22998,목포근대역사관,쇼핑,핑크방
22999,22999,목포근대역사관,쇼핑,아이세이버목포점
23000,23000,목포근대역사관,쇼핑,줄리엣 목포점


In [6]:
df1_=df1.loc[:23002 ,['place_name','tour_element','tour_element_name']]

In [7]:
df1_

,place_name,tour_element,tour_element_name
0,1.21 무장공비 침투로,음식점,0
1,1.21 무장공비 침투로,카페,0
2,1.21 무장공비 침투로,숙소,0
3,1.21 무장공비 침투로,교통,0
4,1.21 무장공비 침투로,공원,0
...,...,...,...
22997,목포근대역사관,쇼핑,산마을생식
22998,목포근대역사관,쇼핑,핑크방
22999,목포근대역사관,쇼핑,아이세이버목포점
23000,목포근대역사관,쇼핑,줄리엣 목포점


### 검색결과 없는 (tour_element_name=0) 인덱스 제거하기

In [8]:
df1__=df1_[df1_['tour_element_name']== '0'].index
df1__=df1_.drop(df1__)
df1__

,place_name,tour_element,tour_element_name
8,1.21 무장공비 침투로,관광명소,연천무장공비침투로
9,1.21 무장공비 침투로,쇼핑,중고피아노고가매입삼익영창야마하
20,송산리고분군,음식점,중동오뎅집
21,송산리고분군,음식점,개성집 공주점
22,송산리고분군,음식점,다래원
...,...,...,...
22997,목포근대역사관,쇼핑,산마을생식
22998,목포근대역사관,쇼핑,핑크방
22999,목포근대역사관,쇼핑,아이세이버목포점
23000,목포근대역사관,쇼핑,줄리엣 목포점


In [9]:
df3=df1__.groupby([df1__.place_name,df1__.tour_element])
df3_=df3.count()

In [10]:
df3_

tour_element_name
place_name    tour_element                   
1.21 무장공비 침투로 관광명소                          1
              쇼핑                            1
관덕정           공원                            2
              관광명소                         13
              교통                           20
...                                       ...
흥국사           쇼핑                           10
              음식점                           9
              주차시설                          1
              카페                            1
              화장실                           2

[374 rows x 1 columns]

In [11]:
#pivot
df3__=df3_.pivot_table(index='place_name', columns='tour_element', values='tour_element_name')
df3__

tour_element,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실
place_name,,,,,,,,,,
1.21 무장공비 침투로,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
관덕정,2.0,13.0,20.0,237.0,237.0,220.0,285.0,80.0,237.0,8.0
광양 중흥사,NaN,NaN,NaN,12.0,6.0,NaN,11.0,NaN,3.0,3.0
구례 사성암,NaN,2.0,20.0,12.0,2.0,NaN,6.0,NaN,3.0,3.0
권금성,1.0,2.0,4.0,10.0,3.0,NaN,16.0,1.0,5.0,1.0
금성산성,NaN,NaN,5.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
김천청암사,NaN,1.0,20.0,2.0,10.0,NaN,8.0,1.0,1.0,1.0
김포장릉,31.0,NaN,21.0,240.0,11.0,92.0,285.0,32.0,237.0,2.0
남사예담촌,NaN,NaN,4.0,8.0,8.0,NaN,9.0,NaN,4.0,1.0


In [12]:
#결측값 0으로 채우기
df_0 = df3__.fillna(0)

In [13]:
df_0

tour_element,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실
place_name,,,,,,,,,,
1.21 무장공비 침투로,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
관덕정,2.0,13.0,20.0,237.0,237.0,220.0,285.0,80.0,237.0,8.0
광양 중흥사,0.0,0.0,0.0,12.0,6.0,0.0,11.0,0.0,3.0,3.0
구례 사성암,0.0,2.0,20.0,12.0,2.0,0.0,6.0,0.0,3.0,3.0
권금성,1.0,2.0,4.0,10.0,3.0,0.0,16.0,1.0,5.0,1.0
금성산성,0.0,0.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
김천청암사,0.0,1.0,20.0,2.0,10.0,0.0,8.0,1.0,1.0,1.0
김포장릉,31.0,0.0,21.0,240.0,11.0,92.0,285.0,32.0,237.0,2.0
남사예담촌,0.0,0.0,4.0,8.0,8.0,0.0,9.0,0.0,4.0,1.0


In [44]:
#정수로 형변환
#df_data=df_0.astype('int')
#df_data

tour_element,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실
place_name,,,,,,,,,,
1.21 무장공비 침투로,0,1,0,1,0,0,0,0,0,0
DMZ 생태평화공원,1,0,2,0,0,0,0,0,0,0
간월암,0,2,48,12,6,0,23,1,6,2
갑곶돈대,2,0,8,114,10,5,85,2,18,2
강릉 경포대,1,5,11,80,233,8,143,8,55,0
...,...,...,...,...,...,...,...,...,...,...
현충사,0,0,20,46,1,1,31,1,5,5
홍유릉,2,1,0,240,3,14,157,10,34,2
화엄사,0,2,4,2,8,3,17,0,5,2


In [14]:
df_0.shape

(54, 10)

## 빅데이터 리뷰 크롤링

### csv파일에서 필요한 데이터 부분만 데이터프레임으로 만들기

In [15]:
df2

,Unnamed: 0,place_name,review
0,0,1.21 무장공비 침투로,0
1,1,서도중앙교회,0
2,2,송산리고분군,454
3,3,관덕정,332
4,4,단양 구인시,895
5,5,보리암,2451
6,6,서울 금선사,32
7,7,남사예담촌,615
8,8,능인선원,17
9,9,정약용유적지,799


In [16]:
df2_=df2.loc[:58,['place_name','review']]

In [17]:
df2_

,place_name,review
0,1.21 무장공비 침투로,0
1,서도중앙교회,0
2,송산리고분군,454
3,관덕정,332
4,단양 구인시,895
5,보리암,2451
6,서울 금선사,32
7,남사예담촌,615
8,능인선원,17
9,정약용유적지,799


In [18]:
Dongjak_Dataframe=pd.merge(df_0,df2_,on='place_name',how='outer')

In [19]:
Dongjak_Dataframe

,place_name,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실,review
0,1.21 무장공비 침투로,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,관덕정,2.0,13.0,20.0,237.0,237.0,220.0,285.0,80.0,237.0,8.0,332
2,광양 중흥사,0.0,0.0,0.0,12.0,6.0,0.0,11.0,0.0,3.0,3.0,16
3,구례 사성암,0.0,2.0,20.0,12.0,2.0,0.0,6.0,0.0,3.0,3.0,797
4,권금성,1.0,2.0,4.0,10.0,3.0,0.0,16.0,1.0,5.0,1.0,487
5,금성산성,0.0,0.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,204
6,김천청암사,0.0,1.0,20.0,2.0,10.0,0.0,8.0,1.0,1.0,1.0,183
7,김포장릉,31.0,0.0,21.0,240.0,11.0,92.0,285.0,32.0,237.0,2.0,0
8,남사예담촌,0.0,0.0,4.0,8.0,8.0,0.0,9.0,0.0,4.0,1.0,615
9,논산 관촉사,4.0,1.0,0.0,240.0,5.0,36.0,285.0,1.0,77.0,3.0,596


In [20]:
#결측값 0으로 채우기
Dongjak_df= Dongjak_Dataframe.fillna(0)

In [21]:
Dongjak_df

,place_name,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실,review
0,1.21 무장공비 침투로,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,관덕정,2.0,13.0,20.0,237.0,237.0,220.0,285.0,80.0,237.0,8.0,332
2,광양 중흥사,0.0,0.0,0.0,12.0,6.0,0.0,11.0,0.0,3.0,3.0,16
3,구례 사성암,0.0,2.0,20.0,12.0,2.0,0.0,6.0,0.0,3.0,3.0,797
4,권금성,1.0,2.0,4.0,10.0,3.0,0.0,16.0,1.0,5.0,1.0,487
5,금성산성,0.0,0.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,204
6,김천청암사,0.0,1.0,20.0,2.0,10.0,0.0,8.0,1.0,1.0,1.0,183
7,김포장릉,31.0,0.0,21.0,240.0,11.0,92.0,285.0,32.0,237.0,2.0,0
8,남사예담촌,0.0,0.0,4.0,8.0,8.0,0.0,9.0,0.0,4.0,1.0,615
9,논산 관촉사,4.0,1.0,0.0,240.0,5.0,36.0,285.0,1.0,77.0,3.0,596


In [22]:
#데이터 합하고, review랑 연결
Dongjak_df.to_csv("result.csv",mode='w')

결과 데이터셋을 보고, 중복된 것은 삭제하고 결과가 이상하게 나온 것은 키워드를 바꿔서 다시 돌린다. 398~457

In [23]:
#중복 행 제거
result=Dongjak_df.drop_duplicates()
result1=result.reset_index(drop=True,inplace=False)
result1

,place_name,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실,review
0,1.21 무장공비 침투로,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,관덕정,2.0,13.0,20.0,237.0,237.0,220.0,285.0,80.0,237.0,8.0,332
2,광양 중흥사,0.0,0.0,0.0,12.0,6.0,0.0,11.0,0.0,3.0,3.0,16
3,구례 사성암,0.0,2.0,20.0,12.0,2.0,0.0,6.0,0.0,3.0,3.0,797
4,권금성,1.0,2.0,4.0,10.0,3.0,0.0,16.0,1.0,5.0,1.0,487
5,금성산성,0.0,0.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,204
6,김천청암사,0.0,1.0,20.0,2.0,10.0,0.0,8.0,1.0,1.0,1.0,183
7,김포장릉,31.0,0.0,21.0,240.0,11.0,92.0,285.0,32.0,237.0,2.0,0
8,남사예담촌,0.0,0.0,4.0,8.0,8.0,0.0,9.0,0.0,4.0,1.0,615
9,논산 관촉사,4.0,1.0,0.0,240.0,5.0,36.0,285.0,1.0,77.0,3.0,596


In [24]:
result1.to_csv("result.csv",mode='w')

In [74]:
#재검색할 키워드 제거
# remove=[]
# remove.append(result1[result1['place_name']== '정지용 생가'].index)
# remove.append(result1[result1['place_name']== '정혜사(순천)'].index)
# remove.append(result1[result1['place_name']== '수도리 전통마을'].index)
# remove.append(result1[result1['place_name']== '김상옥 생가'].index)
# remove.append(result1[result1['place_name']== '개태사'].index)

result2=result1
result2=result2.drop(result2[result2['place_name']== '1.21 무장공비 침투로'].index)
result2=result2.drop(result2[result2['place_name']== '강화 서도 중앙교회'].index)
result2=result2.drop(result2[result2['place_name']== '공주 송산리 고분군과 무령왕릉'].index)
result2=result2.drop(result2[result2['place_name']== '광한루'].index)
result2=result2.drop(result2[result2['place_name']== '관덕정(제주)'].index)
result2=result2.drop(result2[result2['place_name']== '구인사(단양)'].index)
result2=result2.drop(result2[result2['place_name']== '군산 임피역사'].index)
result2=result2.drop(result2[result2['place_name']== '금산 보리암(남해)'].index)
result2=result2.drop(result2[result2['place_name']== '금선사(서울)'].index)
result2=result2.drop(result2[result2['place_name']== '남사예담촌'].index)
result2=result2.drop(result2[result2['place_name']== '다산유적지(남양주)'].index)
result2=result2.drop(result2[result2['place_name']== '능인선원'].index)
result2=result2.drop(result2[result2['place_name']== '단양 금굴 구석기유적지'].index)
result2=result2.drop(result2[result2['place_name']== '담양 금성산성'].index)
result2=result2.drop(result2[result2['place_name']== '담양 명옥헌 원림'].index)
result2=result2.drop(result2[result2['place_name']== '담양 소쇄원'].index)
result2=result2.drop(result2[result2['place_name']== '도동서원'].index)
result2=result2.drop(result2[result2['place_name']== '반구대암각화'].index)
result2=result2.drop(result2[result2['place_name']== '사명대사 생가지'].index)
result2=result2.drop(result2[result2['place_name']== '사성암(구례)'].index)
result2=result2.drop(result2[result2['place_name']== '서봉사(대구)'].index)
result2=result2.drop(result2[result2['place_name']== '설악산 권금성'].index)
result2=result2.drop(result2[result2['place_name']== '수리사(경기)'].index)
result2=result2.drop(result2[result2['place_name']== '수리치골 성지'].index)
result2=result2.drop(result2[result2['place_name']== '수성당'].index)
result2=result2.drop(result2[result2['place_name']== '쌍계사(하동)'].index)
result2=result2.drop(result2[result2['place_name']== '쌍봉사(화순)'].index)
result2=result2.drop(result2[result2['place_name']== '안면암(태안)'].index)
result2=result2.drop(result2[result2['place_name']== '용연사(대구)'].index)
result2=result2.drop(result2[result2['place_name']== '용주사(화성)'].index)
result2=result2.drop(result2[result2['place_name']== '월봉서원'].index)
result2=result2.drop(result2[result2['place_name']== '월정사·월정사 전나무숲'].index)
result2=result2.drop(result2[result2['place_name']== '이성산성'].index)
result2=result2.drop(result2[result2['place_name']== '인헌왕후'].index)
result2=result2.drop(result2[result2['place_name']== '창덕궁과 후원'].index)
result2=result2.drop(result2[result2['place_name']== '창의문(자하문)'].index)
result2=result2.drop(result2[result2['place_name']== '척판암(부산)'].index)
result2=result2.drop(result2[result2['place_name']== '천학정/문암포구'].index)
result2=result2.drop(result2[result2['place_name']== '청룡사(충주)'].index)
result2=result2.drop(result2[result2['place_name']== '포석정'].index)
result2=result2.drop(result2[result2['place_name']== '청암사(김천)'].index)
result2=result2.drop(result2[result2['place_name']== '추사고택'].index)
result2=result2.drop(result2[result2['place_name']== '추사김정희선생고택'].index)
result2=result2.drop(result2[result2['place_name']== '충의사(예산)'].index)
result2=result2.drop(result2[result2['place_name']== '향일암(여수)'].index)
result2=result2.drop(result2[result2['place_name']== '홍유릉'].index)
result2=result2.drop(result2[result2['place_name']== '방화수류정(동북각루)'].index)
result2=result2.drop(result2[result2['place_name']== '흥국사(여수)'].index)
result2=result2.drop(result2[result2['place_name']== '동국사(군산)'].index)
result2=result2.drop(result2[result2['place_name']== '수타사(홍천)'].index)
result2=result2.drop(result2[result2['place_name']== '세연정(완도)'].index)
result2=result2.drop(result2[result2['place_name']== '개평한옥마을'].index)
result2=result2.drop(result2[result2['place_name']== '봉정암(인제)'].index)
result2=result2.drop(result2[result2['place_name']== '관음사(창녕)'].index)
result2=result2.drop(result2[result2['place_name']== '백련사(무주)'].index)
result2=result2.drop(result2[result2['place_name']== '문수암(산청)'].index)
result2=result2.drop(result2[result2['place_name']== '관촉사(논산)'].index)
result2=result2.drop(result2[result2['place_name']== '정혜사(전주)'].index)
result2=result2.drop(result2[result2['place_name']== '오봉산 석굴암'].index)
result2=result2.drop(result2[result2['place_name']== '금용사(대구)'].index)
result2=result2.drop(result2[result2['place_name']== '규봉암(화순)'].index)
result2=result2.drop(result2[result2['place_name']== '희방사(영주)'].index)
result2=result2.drop(result2[result2['place_name']== '흥륜사(인천)'].index)
result2=result2.drop(result2[result2['place_name']== '서암정사(함양)'].index)
result2=result2.drop(result2[result2['place_name']== '신흥사(완도)'].index)
result2=result2.drop(result2[result2['place_name']== '비암사(세종)'].index)
result2=result2.drop(result2[result2['place_name']== '중흥사(광양)'].index)
result2=result2.drop(result2[result2['place_name']== '약천사(제주)'].index)
result2=result2.drop(result2[result2['place_name']== '안국사(무주)'].index)
result2=result2.drop(result2[result2['place_name']== '일붕사(의령)'].index)
result2=result2.drop(result2[result2['place_name']== '법흥사(영월)'].index)
result2=result2.drop(result2[result2['place_name']== '?담양 송강정'].index)
result2=result2.drop(result2[result2['place_name']== '김룡사(문경)'].index)
result2=result2.drop(result2[result2['place_name']== '목포근대역사관 2관'].index)

    
result2_=result2.reset_index(drop=True,inplace=False) #인덱스 재정렬
result2_

,place_name,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실,review
0,5.18 민주광장,7.0,3.0,25.0,225.0,167.0,207.0,270.0,130.0,222.0,8.0,0
1,DMZ 생태평화공원,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
2,각원사,1.0,1.0,20.0,106.0,5.0,12.0,98.0,0.0,53.0,3.0,644
3,간월암,0.0,2.0,48.0,12.0,6.0,0.0,23.0,1.0,6.0,2.0,349
4,갈매못순교성지,0.0,0.0,20.0,46.0,14.0,3.0,43.0,0.0,3.0,1.0,128
...,...,...,...,...,...,...,...,...,...,...,...,...
395,신숭겸장군유적,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,179
396,건봉사,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178
397,문수암 (산청),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
398,고하도이충무공유적,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24


In [75]:
result2_.loc[399,'place_name']

'중명전'

In [76]:
#재검색할 키워드 제거한 크롤링 결과
Result=result2_.iloc[:399]
Result

,place_name,공원,관광명소,교통,쇼핑,숙소,술집,음식점,주차시설,카페,화장실,review
0,5.18 민주광장,7.0,3.0,25.0,225.0,167.0,207.0,270.0,130.0,222.0,8.0,0
1,DMZ 생태평화공원,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
2,각원사,1.0,1.0,20.0,106.0,5.0,12.0,98.0,0.0,53.0,3.0,644
3,간월암,0.0,2.0,48.0,12.0,6.0,0.0,23.0,1.0,6.0,2.0,349
4,갈매못순교성지,0.0,0.0,20.0,46.0,14.0,3.0,43.0,0.0,3.0,1.0,128
...,...,...,...,...,...,...,...,...,...,...,...,...
394,고산 윤선도 유적지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128
395,신숭겸장군유적,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,179
396,건봉사,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178
397,문수암 (산청),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [85]:
#데이터 합하고, review랑 연결
Result.to_csv("result_.csv",mode='w')

In [82]:
place_re=[] #재검색할 키워드

# place_re=[result2_.loc[393+i,'place_name'] for i in range(0,59)]

In [84]:
#다시 검색할 것
place_re.append('1.21 무장공비 침투로')
place_re.append('서도중앙교회')
place_re.append('송산리고분군')
place_re.append('관덕정')
place_re.append('단양 구인시')
place_re.append('보리암')
place_re.append('서울 금선사')
place_re.append('남사예담촌')
place_re.append('능인선원')
place_re.append('정약용유적지')
place_re.append('금성산성')
place_re.append('담양후산리명옥헌원림')
place_re.append('소쇄원')
place_re.append('사명대사유적지관광단지생가지')
place_re.append('구례 사성암')
place_re.append('대구 서봉사')
place_re.append('권금성')
place_re.append('수리사')
place_re.append('하동 쌍계사')                
place_re.append('화순 쌍봉사')
place_re.append('태안 안면암')
place_re.append('대구 용연사')
place_re.append('화성 용주사')
place_re.append('오대산월정사전나무숲길')
place_re.append('김포장릉')
place_re.append('창덕궁 후원')
place_re.append('창의문')
place_re.append('척판암')
place_re.append('천학정')
place_re.append('충주청룡사')
place_re.append('김천청암사')
place_re.append('추사김정희선생고택')
place_re.append('예산 충의사')
place_re.append('여수 향일암')
place_re.append('동북각루')
place_re.append('흥국사')
place_re.append('홍유릉')
place_re.append('동국사')
place_re.append('완도 세연정')
place_re.append('인제 봉정암')
place_re.append('창녕 관음사')
place_re.append('무주 백련사')
place_re.append('산청 문수암')
place_re.append('논산 관촉사')
place_re.append('전주 정혜사')
place_re.append('대구 금용사')
place_re.append('화순 규봉암')
place_re.append('영주 희방사')
place_re.append('인천 흥륜사')
place_re.append('함양 서암정사')
place_re.append('완도 신흥사')
place_re.append('세종 비암사')
place_re.append('광양 중흥사')
place_re.append('제주 약천사')
place_re.append('무주 안국사')
place_re.append('의령 의붕사')
place_re.append('영월 법흥사')
place_re.append('무주 안국사')
place_re.append('목포근대역사관 ')


place_re

['1.21 무장공비 침투로',
 '서도중앙교회',
 '송산리고분군',
 '관덕정',
 '단양 구인시',
 '보리암',
 '서울 금선사',
 '남사예담촌',
 '능인선원',
 '정약용유적지',
 '금성산성',
 '담양후산리명옥헌원림',
 '소쇄원',
 '사명대사유적지관광단지생가지',
 '구례 사성암',
 '대구 서봉사',
 '권금성',
 '수리사',
 '하동 쌍계사',
 '화순 쌍봉사',
 '태안 안면암',
 '대구 용연사',
 '화성 용주사',
 '오대산월정사전나무숲길',
 '김포장릉',
 '창덕궁 후원',
 '창의문',
 '척판암',
 '천학정',
 '충주청룡사',
 '김천청암사',
 '추사김정희선생고택',
 '예산 충의사',
 '여수 향일암',
 '동북각루',
 '흥국사',
 '홍유릉',
 '동국사',
 '완도 세연정',
 '인제 봉정암',
 '창녕 관음사',
 '무주 백련사',
 '산청 문수암',
 '논산 관촉사',
 '전주 정혜사',
 '대구 금용사',
 '화순 규봉암',
 '영주 희방사',
 '인천 흥륜사',
 '함양 서암정사',
 '완도 신흥사',
 '세종 비암사',
 '광양 중흥사',
 '제주 약천사',
 '무주 안국사',
 '의령 의붕사',
 '영월 법흥사',
 '무주 안국사',
 '목포근대역사관 ',
 '1.21 무장공비 침투로',
 '서도중앙교회',
 '송산리고분군',
 '관덕정',
 '단양 구인시',
 '보리암',
 '서울 금선사',
 '남사예담촌',
 '능인선원',
 '정약용유적지',
 '금성산성',
 '담양후산리명옥헌원림',
 '소쇄원',
 '사명대사유적지관광단지생가지',
 '구례 사성암',
 '대구 서봉사',
 '권금성',
 '수리사',
 '하동 쌍계사',
 '화순 쌍봉사',
 '태안 안면암',
 '대구 용연사',
 '화성 용주사',
 '오대산월정사전나무숲길',
 '김포장릉',
 '창덕궁 후원',
 '창의문',
 '척판암',
 '천학정',
 '충주청룡사',
 '김천청암사',
 '추사김정희선생고택',
 '예산 충의사',
 '여수 향일암

In [86]:
#제대로 저장된 장소 텍스트 저장
place_allminusre=[]
place_allminusre=[Result.loc[i,'place_name'] for i in range(len(Result.place_name))]

In [87]:
place_allminusre

['5.18 민주광장',
 'DMZ 생태평화공원',
 '각원사',
 '간월암',
 '갈매못순교성지',
 '감곡매괴성모순례지성당',
 '갑곶돈대',
 '강릉 경포대',
 '강릉 선교장',
 '강성서원',
 '강원감영',
 '강진영랑생가',
 '강천사',
 '강화 고인돌 유적 ',
 '강화 덕진진',
 '강화고려궁지',
 '강화초지진',
 '강화평화전망대',
 '개심사',
 '개암사',
 '경기전',
 '경복궁',
 '경원선 철도중단점',
 '경주 계림',
 '경주 교촌마을',
 '경주 대릉원 일원',
 '경주 독락당',
 '경주 동궁과 월지',
 '경주 문무대왕릉',
 '경주 불국사 ',
 '경주 석굴암 ',
 '경주 양동마을 ',
 '경주 오릉',
 '경주 첨성대',
 '경주역사유적지구',
 '경희궁',
 '계룡산도예촌',
 '계백장군 유적지',
 '고령 대가야유적지',
 '고모산성',
 '고성 문암리 유적',
 '고성 왕곡마을',
 '고성화암사',
 '고운사',
 '고창 고인돌 유적',
 '고창 무장현 관아와 읍성',
 '고창읍성',
 '골굴사',
 '공주 공산성 ',
 '관수옥과 초량왜관 터',
 '관음사(대구)',
 '관음사(무주)',
 '광성보',
 '광한루원',
 '광화문',
 '구 보성여관',
 '구례 운조루 고택',
 '구룡사',
 '구산성지',
 '국립4.19민주묘지',
 '국립대전현충원',
 '국립서울현충원',
 '국채보상운동기념공원',
 '군산 신흥동 일본식가옥',
 '군위 아미타여래삼존 석굴',
 '금당실마을',
 '금둔사',
 '금정산성',
 '길상사(서울)',
 '김유정 문학촌',
 '김제 벽골제',
 '김제금산사',
 '김포 덕포진',
 '나주 금성관',
 '나주 목사내아',
 '낙산사',
 '낙산사 의상대',
 '낙화암',
 '남계서원 ',
 '남도한옥민박마을',
 '남선창고터',
 '남양성모성지',
 '남양주 광릉',
 '남원 만인의총',
 '남해 관음포 이충무공 유적',
 '내소사',
 '노동당사',
 '노무현 대통령 생가',
 

In [88]:
len(place_allminusre)

399

In [89]:
def write_txt(list,fname,sep):
    file=open(fname,'w')
    vstr=""

    for a in list:
        vstr = vstr+ str(a) + sep
    
    vstr =vstr.rstrip(sep)
    
    file.writelines(vstr)
    file.close()
    print('저장완료')

In [90]:
write_txt(place_allminusre,"place_allminusre500~1000.txt",sep='\n')

저장완료


In [120]:
write_txt(place_re,"place_re500~1000.txt",sep='\n')

저장완료


In [91]:
len(place_re)

118